In [33]:
import numpy as np
import pandas as pd
import ast

In [34]:
movies=pd.read_csv("./dataset/tmdb_5000_movies.csv")
credits=pd.read_csv("./dataset/tmdb_5000_credits.csv")

In [35]:
# merging both dataset

df=movies.merge(credits,on="title")

## EDA ##

In [36]:
df.isna().sum()

budget                     0
genres                     0
homepage                3096
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
movie_id                   0
cast                       0
crew                       0
dtype: int64

### Remove unnecessary columns ###

In [37]:
df=df[["movie_id","title","overview","genres","keywords","cast","crew"]]

In [38]:
df.dropna(inplace=True)

In [39]:
df.isna().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [40]:
df["genres"][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [41]:
def convert(obj):
    L_genre=[]
    for i in ast.literal_eval(obj):
        L_genre.append(i["name"])
    return L_genre

In [42]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i["name"])
            break
    return L

In [43]:
df["genres"]=df["genres"].apply(convert)

In [44]:
df["keywords"]=df["keywords"].apply(convert)

In [45]:
df["cast"]=df["cast"].apply(convert).apply(lambda x:x[:3])

In [46]:
df["crew"]=df["crew"].apply(fetch_director)

In [47]:
df["overview"]=df["overview"].apply(lambda x:x.split())

Removing spaces between the names to avoid confusion for recommender systems

For example

Sam Mendes -> SamMendis
Sam Worthington ->SamWorthington

otherwise system gets confused which sam are we looking for

In [48]:
def remove_spaces(obj):
    return obj.apply(lambda x:[i.replace(" ","") for i in x])

In [49]:
df["genres"]=remove_spaces(df["genres"])
df["keywords"]=remove_spaces(df["keywords"])
df["cast"]=remove_spaces(df["cast"])
df["crew"]=remove_spaces(df["crew"])

In [50]:
df["tags"]=df["overview"]+df["genres"]+df["keywords"]+df["cast"]+df["crew"]

In [51]:
df=df[["movie_id","title","tags"]]

In [52]:
df["tags"]=df["tags"].apply(lambda x:" ".join(x))

In [53]:
df["tags"]=df["tags"].apply(lambda x:x.lower())

In [54]:
df["tags"][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

we have converted tags now. 
to find similiarity between movies, we find similiarty between tags

convert tags to vector using bag of words and perform vectorization without stop words



In [55]:
df.info()

<class 'pandas.DataFrame'>
Index: 4806 entries, 0 to 4808
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   movie_id  4806 non-null   int64
 1   title     4806 non-null   str  
 2   tags      4806 non-null   str  
dtypes: int64(1), str(2)
memory usage: 279.2 KB


## Vectorization ##

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(max_features=5000, stop_words="english", ngram_range=(1,2), min_df=2)
vectors = tfidf.fit_transform(df["tags"])



In [57]:
# 1) Concatenate all tags
# 2) Take 5000 most common words not including stop words
# 3) create a vector for each movie with count of each word occuring in that movie

In [58]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv=CountVectorizer(max_features=5000,stop_words='english')
# vectors=cv.fit_transform(df["tags"]).toarray()

## Stemming ##

We do stemming now to avoid words like (action actions), (love loved loving) to be counted as different.

In [59]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [60]:
def stem(txt):
    y=[]
    for i in txt.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [61]:
df["tags"]=df["tags"].apply(stem)

In [63]:
from sklearn.metrics.pairwise import cosine_similarity

In [64]:
similarity=cosine_similarity(vectors)

In [65]:
def recommend(movie):
    #get index of movie
    
    movie_index=df[df['title']==movie].index[0]
    movie_distance=similarity[movie_index]
    movies_list=sorted(list(enumerate(movie_distance)), reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(df.iloc[i[0]].title)

In [67]:
recommend('Avatar')

Star Trek Into Darkness
Falcon Rising
Battle: Los Angeles
Galaxina
Titan A.E.


In [ ]:
similarity[1216]

array([0.02341465, 0.10369517, 0.        , ..., 0.0860663 , 0.15975241,
       0.09365858], shape=(4806,))